In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Load your dataset
from google.colab import files
uploaded = files.upload()

Saving RCA_V3.csv to RCA_V3.csv


In [ ]:

data = (pd.read_csv('RCA_V3.csv',sep=None,engine='python', usecols = ['RCA_Main_Root_Cause','RCA Defect Domain'])).dropna(how='all')


# Inspect the dataframe columns
print("Data columns:", data.columns)

# Rename columns for consistency
data.columns = ['text', 'label']

# Ensure the column names are correct
print("Renamed data columns:", data.columns)

# Check for NaN values
print("Number of NaN values in text column:", data['text'].isna().sum())
print("Number of NaN values in label column:", data['label'].isna().sum())

# Drop NaN values if any
data = data.dropna(subset=['text', 'label'])

# Preprocess the text data
texts = data['text'].astype(str).values  # Convert all entries to strings

#labels = data['label'].values
# Label encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['label'].astype(str).values)  # Convert labels to numeric



Data columns: Index(['RCA_Main_Root_Cause', 'RCA Defect Domain'], dtype='object')
Renamed data columns: Index(['text', 'label'], dtype='object')
Number of NaN values in text column: 316
Number of NaN values in label column: 264


In [ ]:
# Ensure all labels are integers
print("Encoded labels:", labels)
print("Unique labels:", np.unique(labels))

Encoded labels: [0 0 2 ... 4 0 3]
Unique labels: [0 1 2 3 4 5 6 7 8]


In [ ]:
# Tokenize the text data
max_features = 20000
maxlen = 80  # cut texts after this number of words
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)


In [ ]:
# Pad sequences
x_data = pad_sequences(sequences, maxlen=maxlen)


In [ ]:
# Convert labels to numpy array
y_data = np.array(labels)

In [ ]:
# Verify data types
print(f"x_data type: {x_data.dtype}, y_data type: {y_data.dtype}")
print(f"x_data shape: {x_data.shape}, y_data shape: {y_data.shape}")

x_data type: int32, y_data type: object
x_data shape: (10521, 80), y_data shape: (10521,)


In [ ]:
# Ensure there are no float types in y_data
print(f"y_data type after conversion: {y_data.dtype}")

y_data type after conversion: object


In [ ]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)


In [ ]:
# Verify data types
print(f"x_data type: {x_data.dtype}, y_data type: {y_data.dtype}")
print(f"x_data shape: {x_data.shape}, y_data shape: {y_data.shape}")

x_data type: int32, y_data type: object
x_data shape: (10521, 80), y_data shape: (10521,)


In [ ]:
# Ensure training and testing sets are of correct types
print(f"x_train type: {x_train.dtype}, y_train type: {y_train.dtype}")
print(f"x_test type: {x_test.dtype}, y_test type: {y_test.dtype}")

x_train type: int32, y_train type: object
x_test type: int32, y_test type: object


In [ ]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:



# Drop unnecessary columns and rename cols,
# Assuming we need only 'text' and 'label' columns
data = (pd.read_csv('RCA_V3.csv',sep=None,engine='python', usecols = ['RCA_Main_Root_Cause','RCA Defect Domain'])).dropna(how='all')
data.columns = ['text', 'label']
data.head()
print(data)

#Check for missing values
missing_values = data.isnull().sum()


# Step 6: Fill NaN in 'text' column with an empty string
data['text'].fillna('', inplace=True)

# Step 7: Create Feature and Label sets
X = data['text']
y = data['label']

# Step 8: Assuming 'text' column may have NaN values, fill NaN with an empty string (Redundant)
X.fillna('', inplace=True)

# Step 9: Encode labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 10: Train-test split (67% train - 33% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.33, random_state=42)

# Step 11: Train Bag of Words model
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Step 12: Reverse the encoding to get original class labels
original_labels = label_encoder.inverse_transform(y_encoded)

# Step 13: Tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_len = max(len(seq) for seq in X_train_seq)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Step 14: Convert labels to one-hot encoding using numpy
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

# Step 15: Define RNN
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train_pad, y_train_oh, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test_oh)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


                                                    text        label
0      Parking position handling was not well address...       Design
1                       The SDS of WH was not up-to-date       Design
2                 Direct use of a trace definition file.  Integration
3                 Patch too old, not traceable any more.  Integration
4      Clearmake does not remove already build obsole...  Integration
...                                                  ...          ...
10735                                         SW design.       Design
10736                                         SW design.       Design
10737  A DDF timeout is not reported by the CN facili...  Realization
10738  Automated regression does not capture the impa...       Design
10739  The combination of file sizes, number of scann...        Other

[10521 rows x 2 columns]
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape     

In [ ]:
# Evaluate the model
score, acc = model.evaluate(X_test_pad, y_test_oh, batch_size=32)
print("Test score:", score)
print("Test accuracy:", acc)

109/109 [==============================] - 10s 90ms/step - loss: 1.7546 - accuracy: 0.5259
Test score: 1.7546255588531494
Test accuracy: 0.525921642780304
